# Document Reference Extraction

This notebook extracts and matches references to paragraphs, annexes, figures, and tables from legal document text.

In [1]:
import json
import re
import pandas as pd
from typing import List, Dict, Set

## Load Data

In [2]:
# Load evaluation and test data
with open("evaluation_data.json", "r") as f:
    eval_data = json.load(f)

with open("test_data.json", "r") as f:
    test_data = json.load(f)

# Create DataFrames
df_eval = pd.DataFrame(eval_data["paragraphLinks"])
df_test = pd.DataFrame(test_data["paragraphLinks"])

print(f"Evaluation data: {len(df_eval)} paragraphs")
print(f"Paragraphs with references: {df_eval['targetIds'].apply(lambda x: len(x) > 0).sum()}")
df_eval.head()

Evaluation data: 941 paragraphs
Paragraphs with references: 81


,text,id,targetIds
0,Preamble,659d4ec2cbbf0962d357384e,[]
1,The World Forum for Harmonization of Vehicle R...,659d50422e63b7837a047b7c,[]
2,1 TRANS/WP.29/1045 as amended by ECE/TRANS/WP....,659d50452e63b7837a047b84,[]
3,Introduction,659d501f2e63b7837a047b6b,[]
4,The text hereafter updates the recommendations...,659d50872e63b7837a047ba6,"[659d4ec2cbbf0962d3573947, 659d4ec2cbbf0962d35..."


## Textual ID Extraction

In [3]:
def generate_textual_id(text: str) -> str:
    """Extract textual ID from paragraph text (e.g., 'paragraph 1.2.3.', 'Annex 4')."""
    # Try paragraph numeric id at start (e.g., "1.2.3. Some text..." or "1.2.3 Some text...")
    m = re.match(r'^(\d+(?:\.\d+)*\.?)(\.|\s)', text)
    if m:
        number_part = m.group(1)
        # Ensure trailing dot is preserved
        if not number_part.endswith('.'):
            number_part += '.'
        return f"paragraph {number_part}"
    
    # Try other label types at start (e.g., "Annex 4", "Figure 1")
    for prefix in ["Annex", "Figure", "Table"]:
        m = re.match(rf'^{prefix}\s+[A-Z0-9]+', text, flags=re.IGNORECASE)
        if m:
            return m.group(0)
    
    return None

def build_textual_id_map(text_series: pd.Series, id_series: pd.Series) -> Dict[str, str]:
    """Build mapping from textual IDs to document IDs."""
    df_temp = pd.DataFrame({'text': text_series, 'id': id_series})
    df_temp['textualId'] = df_temp['text'].apply(generate_textual_id)
    df_filtered = df_temp[df_temp['textualId'].notnull()]
    return dict(zip(df_filtered['textualId'], df_filtered['id']))

# Create textual ID mapping
df_eval['textualId'] = df_eval['text'].apply(generate_textual_id)
textual_to_id = build_textual_id_map(df_eval["text"], df_eval["id"])
print(f"Created {len(textual_to_id)} textual ID mappings")

Created 625 textual ID mappings


## Reference Extraction

In [4]:
def extract_references(text: str) -> List[str]:
    """Extract references like 'paragraph 1.2', 'Annex 4', 'paragraphs 1.1 to 1.3'."""
    ref_types = r"(?:annex|annexes|paragraph|paragraphs|section|sections|table|tables|figure|figures)"
    number_pattern = r"\d+(?:\.\d+)*\.?"

    # Range pattern (e.g., "paragraphs 4. to 7.")
    range_pattern = rf"({ref_types})\s+({number_pattern})\s+((?:to|and)\s+{number_pattern})"
    # Simple pattern (e.g., "paragraph 1.2")
    simple_pattern = rf"({ref_types})\s+({number_pattern})"

    matches = []
    range_positions = []

    # Find paragraph range starts for subparagraph matching
    paragraph_range_starts = set()
    for match in re.finditer(range_pattern, text, re.IGNORECASE):
        if match.start() == 0:  # Skip matches at text beginning
            continue
        matches.append(match.group(0))
        range_positions.append((match.start(), match.end()))

        ref_type = match.group(1).lower()
        if "paragraph" in ref_type:
            start_num = match.group(2).rstrip(".")
            paragraph_range_starts.add(start_num)

    # Find simple matches
    for match in re.finditer(simple_pattern, text, re.IGNORECASE):
        if match.start() == 0:  # Skip matches at text beginning
            continue

        # Skip overlaps with ranges
        overlaps = any(
            match.start() < end and match.end() > start
            for start, end in range_positions
        )
        if overlaps:
            continue

        ref_type = match.group(1).lower()
        number = match.group(2).rstrip(".")

        # FIXED: For paragraphs, include all if no ranges, or only matching ones if ranges exist
        if "paragraph" in ref_type:
            if not paragraph_range_starts or any(number == start or number.startswith(start + ".") for start in paragraph_range_starts):
                matches.append(match.group(0))
        else:
            matches.append(match.group(0))

    return matches
# Apply reference extraction
df_eval['references'] = df_eval['text'].apply(extract_references)
print(f"Extracted references from {df_eval['references'].apply(len).gt(0).sum()} paragraphs")

Extracted references from 95 paragraphs


In [5]:
df_eval[df_eval['textualId'] == "paragraph 2.5.1"]
# df_eval[df_eval['references'].apply(lambda refs: "paragraph 2.5.1" in refs)]

,text,id,targetIds,textualId,references


## Reference Matching

In [6]:
def is_section_in_range(section: str, start: str, end: str) -> bool:
    """Check if a section number is within a given range."""
    return start <= section <= end

def find_matching_ids(references: List[str], key_to_id_dict: Dict[str, str]) -> List[str]:
    """Find matching document IDs for extracted references."""
    matching_ids = set()

    for ref in references:
        ref = ref.strip()

        # Handle "and" pattern (e.g., "Paragraphs 1. and 2.")
        and_match = re.search(r'(.+?)\s+([\d.]+)\s+and\s+([\d.]+)', ref, re.IGNORECASE)
        if and_match:
            prefix = and_match.group(1).rstrip('s').strip()
            for num in [and_match.group(2).rstrip('.'), and_match.group(3).rstrip('.')]:
                key = f"{prefix} {num}"
                if key in key_to_id_dict:
                    matching_ids.add(key_to_id_dict[key])
            continue

        # Handle "to" pattern (e.g., "paragraphs 4. to 7.")
        range_match = re.search(r'(.+?)\s+([\d.]+)\s+to\s+([\d.]+)', ref, re.IGNORECASE)
        if range_match:
            prefix = range_match.group(1).rstrip('s').strip()
            start = range_match.group(2).rstrip('.')
            end = range_match.group(3).rstrip('.')

            # Find all matching keys in range
            for key in key_to_id_dict:
                key_match = re.match(rf'{prefix}\s+([\d.]+)', key, re.IGNORECASE)
                if key_match:
                    section = key_match.group(1)
                    if is_section_in_range(section, start, end):
                        matching_ids.add(key_to_id_dict[key])
            continue

        # Handle exact match
        if ref in key_to_id_dict:
            matching_ids.add(key_to_id_dict[ref])

    return list(matching_ids)

# Apply reference matching
df_eval['targetIdsPredicted'] = df_eval['references'].apply(
    lambda refs: find_matching_ids(refs, textual_to_id)
)

print(f"Generated predictions for {len(df_eval)} paragraphs")

Generated predictions for 941 paragraphs


## Evaluation

In [7]:
def evaluate_accuracy_and_recall(df: pd.DataFrame) -> Dict[str, float]:
    """Calculate accuracy and recall metrics."""
    total_accuracy = 0
    total_recall = 0
    count = 0

    for _, row in df.iterrows():
        true_ids = set(row["targetIds"])
        pred_ids = set(row["targetIdsPredicted"])

        tp = len(true_ids & pred_ids)
        fp = len(pred_ids - true_ids)
        fn = len(true_ids - pred_ids)

        # Calculate metrics
        denom_accuracy = tp + fp + fn
        denom_recall = tp + fn

        row_accuracy = tp / denom_accuracy if denom_accuracy > 0 else 1.0
        row_recall = tp / denom_recall if denom_recall > 0 else 1.0

        total_accuracy += row_accuracy
        total_recall += row_recall
        count += 1

    return {
        "accuracy": total_accuracy / count if count > 0 else 0,
        "recall": total_recall / count if count > 0 else 0
    }

# Evaluate results
metrics = evaluate_accuracy_and_recall(df_eval)
print(f"Accuracy: {metrics['accuracy']:.2%}")
print(f"Recall: {metrics['recall']:.2%}")

Accuracy: 94.07%
Recall: 95.38%


## Error Analysis (Sample)

In [8]:
def show_prediction_errors(df: pd.DataFrame, id_to_text: Dict[str, str], max_examples: int = 3):
    """Show examples of prediction errors for debugging."""
    count = 0
    
    for idx, row in df.iterrows():
        if count >= max_examples:
            break
            
        true_ids = set(row["targetIds"])
        pred_ids = set(row["targetIdsPredicted"])
        fp = pred_ids - true_ids  # False positives
        fn = true_ids - pred_ids  # False negatives

        if fp or fn:  # Only show errors
            print(f"\n--- Row {idx} ---")
            print(f"Text: {row['text'][:200]}")
            print(f"References found: {row['references']}")
            
            if fn:
                print("❌ Missed (False Negatives):")
                for id_ in list(fn)[:2]:  # Show only first 2
                    ref_text = id_to_text.get(id_, '[Unknown]')
                    textual_id = generate_textual_id(ref_text)
                    print(f"  - {textual_id}: {ref_text[:100]}...")
            
            if fp:
                print("❌ Incorrect Predictions (False Positives):")
                for id_ in list(fp)[:2]:  # Show only first 2
                    ref_text = id_to_text.get(id_, '[Unknown]')
                    textual_id = generate_textual_id(ref_text)
                    print(f"  - {textual_id}: {ref_text[:1000]}...")
            
            count += 1

# Create ID to text mapping for error analysis
id_to_text = dict(zip(df_eval['id'], df_eval['text']))

# Show sample errors
show_prediction_errors(df_eval, id_to_text, max_examples=10)


--- Row 4 ---
Text: The text hereafter updates the recommendations of the Consolidated Resolution on the Construction of Vehicles and provides information on the legal texts under the framework of the 1958 Agreement (UN 
References found: ['Paragraphs 1. and 2.', 'paragraphs 4. to 7.', 'Annex 3', 'Annex 4', 'Annex 5', 'Annex 6', 'Annex 7']
❌ Missed (False Negatives):
  - Annex 5: Annex 5 Design principles for Control Systems of Advanced Driver Assistance System (ADAS)...
  - paragraph 8.: 8. Recommendations...
❌ Incorrect Predictions (False Positives):
  - paragraph 4.14.1.: 4.14.1. The coordinates of the "H" point are measured with respect to the three-dimensional reference system....
  - paragraph 5.: 5. References Bainbridge, L. (1987). Ironies of Automation. In J. Rasmussen, K. Duncan, and J. Leplat (Eds.), New Technology and Human Error. Chichester and New York: John Wiley & Sons. Brook-Carter, N. & Parkes, A. (2000). ADAS and Driver Behavioural Adaptation. European Community: Co


--- Row 80 ---
Text: 2.8.1. Definition. Off-road vehicles are considered to be the vehicles of categories M and N satisfying the requirements of this paragraph, checked under the conditions indicated in paragraphs 2.8.2. 
References found: ['paragraphs 2.8.2. and 2.8.3.']
❌ Missed (False Negatives):
  - paragraph 2.8.2.: 2.8.2. Load and checking conditions...
  - paragraph 2.8.3.: 2.8.3. Definitions and sketches of front and rear incidence angles, ramp angle and ground clearance....

--- Row 86 ---
Text: 2.8.2.2. Power-driven vehicles other than those referred to in paragraph 2.8.2.1. shall be loaded to the technically permissible maximum mass stated by the manufacturer.
References found: ['paragraph 2.8.2.1.']
❌ Incorrect Predictions (False Positives):
  - paragraph 2.8.2.1.: 2.8.2.1. Vehicles in category N1 with a maximum mass not exceeding 2,000 kg and vehicles in category M1 shall be in running order, namely with coolant fluid, lubricants, fuel, tools, spare-wheel and a driver con

## Apply to Test Data

In [13]:
def process_test_data(df_test: pd.DataFrame, textual_to_id: Dict[str, str]) -> pd.DataFrame:
    """Apply the same processing pipeline to test data."""
    df_test = df_test.copy()
    
    # Extract textual IDs and references
    df_test['textualId'] = df_test['text'].apply(generate_textual_id)
    df_test['references'] = df_test['text'].apply(extract_references)
    
    # Match references to IDs
    df_test['targetIdsPredicted'] = df_test['references'].apply(
        lambda refs: find_matching_ids(refs, textual_to_id)
    )
    
    return df_test

# Process test data
df_test_processed = process_test_data(df_test, textual_to_id)

print(f"Processed {len(df_test_processed)} test paragraphs")
print(f"Predictions generated for {df_test_processed['targetIdsPredicted'].apply(len).gt(0).sum()} paragraphs")

# Show sample results
sample_with_predictions = df_test_processed[df_test_processed['targetIdsPredicted'].apply(len) > 0].head(3)
for _, row in sample_with_predictions.iterrows():
    print(f"\nText: {row['text']}...")
    print(f"References: {row['references']}")
    print(f"Predicted IDs: {len(row['targetIdsPredicted'])} items")

# Save processed test data
df_test_processed.to_csv("test_data_with_predictions.csv", index=False)

Processed 298 test paragraphs
Predictions generated for 17 paragraphs

Text: 2.14. "Sufficient nominal Peak Braking Coefficient (PBC)": means a road surface friction coefficient of: (a) 0.9, when measured using the American Society for Testing and Materials (ASTM) of E1136-19 standard reference test tyre in accordance with ASTM Method E1337-19 at a speed of 40 mph; (b) 1.017, when measured using either: (i) The American Society for Testing and Materials (ASTM) of F2493-20 standard reference test tyre in accordance with ASTM Method E1337‑19 at a speed of 40 mph; or (ii) The k-test method specified in Appendix 2 to Annex 6 of UN Regulation No. 13-H. (c) The required value to permit the design maximum deceleration of the relevant vehicle, when measured using the k-test method in Appendix 2 to Annex 13 of UN Regulation No. 13....
References: ['Annex 6', 'Annex 13']
Predicted IDs: 1 items

Text: 4.3. Notice of approval or of refusal or withdrawal of approval pursuant to this Regulation shal

## Points for Improvement

### Current Achievement
- **Time Investment**: 3-4 hours for rule-based solution
- **Cost**: Zero (vs. paid ML/API solutions)
- **Accuracy**: ~94-95% with simple regex patterns
- **Approach**: Practical, explainable, and debuggable

### Technical Improvements

#### 1. Duplicate Paragraph Numbers
- **Issue**: Multiple paragraphs with same number exist in document (e.g., two different "paragraph 6.")
- **Example**:
❌ Missed: paragraph 6. → "6. Requirements for the protection of the environment..."
❌ False Positive: paragraph 6. → "6. Appendices content Appendix 1 shows..."

#### 2. Cross-Document Reference Handling
- **Issue**: References to UN Regulations (e.g., 'UN Regulation No. 13, Annex 4') not resolved
- **Solution**: Build cross-document mapping or external document database
- **Impact**: Handle regulatory cross-references better

#### 3. Local LLM Implementation
- **Approach**: Deploy local LLM (Llama 3, Mixtral, etc.) instead of paid APIs
- **Benefits**:
  - **Privacy**: No data sent to external services
  - **Cost**: One-time setup vs. per-request pricing  
  - **Customization**: Fine-tune on legal document patterns
  - **Contextual Understanding**: Better handling of ambiguous references

### Data Quality Issues

#### 1. Training Annotation Errors
- **Issue**: Ground truth labels might be inconsistent or incorrect
- **Example**: Text mentions "paragraphs 2.8.2. and 2.8.3." but system finds `['paragraphs 2.8.2. and 2.8.3.']` - this should be a **correct match**, not a false negative
- **Impact**: Evaluation metrics may be artificially low due to annotation errors
- **Solution**: 
  - Manual review of "false negatives" to identify annotation mistakes
  - Re-annotate subset of data for quality assessment
  - Use multiple annotators with inter-annotator agreement scoring
